
# Part B – Sentiment vs Trader Behavior Analysis

This notebook answers:

1. Does performance (PnL, win rate, drawdown proxy) differ between Fear vs Greed days?
2. Do traders change behavior based on sentiment?
3. Segment analysis:
   - High vs Low leverage traders
   - Frequent vs Infrequent traders
   - Consistent vs Inconsistent traders
4. Insights backed by charts and tables


In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load datasets
trades = pd.read_csv('historical_data.csv')
sentiment = pd.read_csv('fear_greed_index.csv')

# Standardize columns
trades.columns = trades.columns.str.lower().str.strip()
sentiment.columns = sentiment.columns.str.lower().str.strip()

# Detect date columns
trade_date = [c for c in trades.columns if 'date' in c or 'time' in c][0]
sent_date = [c for c in sentiment.columns if 'date' in c or 'time' in c][0]

trades['date'] = pd.to_datetime(trades[trade_date], errors='coerce').dt.date
sentiment['date'] = pd.to_datetime(sentiment[sent_date], errors='coerce').dt.date

# Rename size tokens column to trade_size
if 'size tokens' in trades.columns:
    trades.rename(columns={'size tokens':'trade_size'}, inplace=True)

# Detect numeric columns
num_cols = trades.select_dtypes(include=np.number).columns.tolist()
pnl_col = num_cols[0]
trades.rename(columns={pnl_col:'pnl'}, inplace=True)

# Sentiment classification column
sent_col = [c for c in sentiment.columns if 'class' in c or 'fear' in c or 'greed' in c][0]
sentiment.rename(columns={sent_col:'classification'}, inplace=True)

# Merge
data = pd.merge(trades, sentiment[['date','classification']], on='date', how='left')

# Win flag
data['win'] = np.where(data['pnl'] > 0, 1, 0)


In [4]:
fear_greed = data.groupby('classification').agg(
    avg_pnl=('pnl','mean'),
    win_rate=('win','mean'),
    drawdown_proxy=('pnl', lambda x: (x < 0).mean()),
    trades=('pnl','count')
)

print("Fear vs Greed Performance Summary:")
print(fear_greed)

if fear_greed.empty:
    print("No data available after grouping. Check 'classification' column.")
else:
    # Chart 1: Avg PnL
    plt.figure()
    fear_greed['avg_pnl'].plot(kind='bar')
    plt.title("Average PnL by Sentiment")
    plt.show()

    # Chart 2: Win Rate
    plt.figure()
    fear_greed['win_rate'].plot(kind='bar')
    plt.title("Win Rate by Sentiment")
    plt.show()


Fear vs Greed Performance Summary:
Empty DataFrame
Columns: [avg_pnl, win_rate, drawdown_proxy, trades]
Index: []
No data available after grouping. Check 'classification' column.


In [ ]:
import matplotlib.pyplot as plt

# Ensure classification column exists
if 'classification' not in data.columns:
    print("Column 'classification' not found.")
else:
    # Drop rows where classification is NaN
    clean_data = data.dropna(subset=['classification'])

    if clean_data.empty:
        print("No data available after removing NaNs.")
    else:
      
        trade_freq = clean_data.groupby('classification').size()

        if not trade_freq.empty:
            plt.figure()
            trade_freq.plot(kind='bar')
            plt.title("Trade Frequency by Sentiment")
            plt.tight_layout()
            plt.show()
        else:
            print("Trade frequency is empty.")

        
        # Average Trade Size
     
        if 'trade_size' in clean_data.columns:
            trade_size = clean_data.groupby('classification')['trade_size'].mean()

            if not trade_size.empty:
                plt.figure()
                trade_size.plot(kind='bar')
                plt.title("Average Trade Size by Sentiment")
                plt.tight_layout()
                plt.show()
            else:
                print("Trade size grouping is empty.")

       
        # Average Leverage
        
        if 'leverage' in clean_data.columns:
            leverage = clean_data.groupby('classification')['leverage'].mean()

            if not leverage.empty:
                plt.figure()
                leverage.plot(kind='bar')
                plt.title("Average Leverage by Sentiment")
                plt.tight_layout()
                plt.show()
            else:
                print("Leverage grouping is empty.")


No data available after removing NaNs.


In [7]:

# High vs Low leverage segment
if 'leverage' in data.columns:
    median_lev = data['leverage'].median()
    data['lev_segment'] = np.where(data['leverage'] > median_lev,
                                   'High_Leverage','Low_Leverage')

# Frequent vs Infrequent traders
trader_col = [c for c in data.columns if 'trader' in c or 'account' in c or 'user' in c]

if trader_col:
    trader_col = trader_col[0]
    trade_count = data.groupby(trader_col).size()
    median_trades = trade_count.median()
    data['freq_segment'] = data[trader_col].map(
        lambda x: 'Frequent' if trade_count[x] > median_trades else 'Infrequent'
    )

# Consistency segment
if trader_col:
    pnl_std = data.groupby(trader_col)['pnl'].std()
    median_std = pnl_std.median()
    data['consistency_segment'] = data[trader_col].map(
        lambda x: 'Consistent' if pnl_std[x] < median_std else 'Inconsistent'
    )

# Segment Performance Table
if 'lev_segment' in data.columns:
    seg_perf = data.groupby(['classification','lev_segment'])['pnl'].mean()
    print("Segment Performance (Leverage):")
    print(seg_perf)


In [8]:

print("INSIGHT 1: Compare avg_pnl between Fear and Greed days from summary table.")
print("INSIGHT 2: Observe leverage and trade size charts for behavior shifts.")
print("INSIGHT 3: Compare High vs Low leverage segment performance.")


INSIGHT 1: Compare avg_pnl between Fear and Greed days from summary table.
INSIGHT 2: Observe leverage and trade size charts for behavior shifts.
INSIGHT 3: Compare High vs Low leverage segment performance.
